In [2]:
import torch

a = torch.load('./coref_elmo_dir/best.th')
# print(type(a))

# print(sorted([len(a[i]) for i in a.keys()]))

# print([i for i in a.keys() if len(a[i]) == 1024])
# # print([i for i in a.keys() if len(a[i]) == 1])

names = ['_text_field_embedder.token_embedder_elmo._elmo.scalar_mix_0.gamma', '_text_field_embedder.token_embedder_elmo._elmo.scalar_mix_0.scalar_parameters.0', '_text_field_embedder.token_embedder_elmo._elmo.scalar_mix_0.scalar_parameters.1', '_text_field_embedder.token_embedder_elmo._elmo.scalar_mix_0.scalar_parameters.2']

weights = [float(a[i]) for i in names]

from allennlp.commands.elmo import ElmoEmbedder
elmo = ElmoEmbedder()


12/16/2018 16:49:10 - INFO - allennlp.commands.elmo -   Initializing ELMo.


In [3]:
# vectors = elmo.embed_sentence(["Richard", "loves", "himself", "so", "much", "that", "he", "made", "love", "to", "himself"])
# orig_sentence = "dogs ran around the yard . they ran quickly . bananas taste good .".split(" ")

orig_sentence = ["In", "the", "same", "way", "that", "no", "country", "has", "a", "law", "against", "cannibals", "eating", "its", "prime", "minister", ",", "because", "such", "an", "act", "is", "unthinkable", ",", "international", "law", "does", "not", "address", "killers", "shooting", "from", "hospitals", ",", "mosques", "and", "ambulances", ",", "while", "being", "protected", "by", "their", "Government", "or", "society", ".", "International", "law", "does", "not", "know", "how", "to", "handle", "someone", "who", "sends", "children", "to", "throw", "stones", ",", "stands", "behind", "them", "and", "shoots", "with", "immunity", "and", "can", "not", "be", "arrested", "because", "he", "is", "sheltered", "by", "a", "Government", ".", "International", "law", "does", "not", "know", "how", "to", "deal", "with", "a", "leader", "of", "murderers", "who", "is", "royally", "and", "comfortably", "hosted", "by", "a", "country", ",", "which", "pretends", "to", "condemn", "his", "acts", "or", "just", "claims", "to", "be", "too", "weak", "to", "arrest", "him", "."]
vectors = elmo.embed_sentence(orig_sentence)

In [4]:
import numpy as np
import scipy

val1 = weights[0]*np.add(np.add(weights[1]*vectors[0][1], weights[2]*vectors[1][1]), weights[3]*vectors[2][1])

val2 = weights[0]*np.add(np.add(weights[1]*vectors[0][7], weights[2]*vectors[1][7]), weights[3]*vectors[2][7])

dist = scipy.spatial.distance.cosine(val1, val2) # cosine distance between "apple" and "carrot" in the last layer
dist2 = scipy.spatial.distance.cosine(vectors[2][1], vectors[2][7])
print(dist)
print(dist2)
print(len(vectors[0]))
# 0.18020617961883545

0.7737881392240524
0.8022511005401611
123


In [5]:
elmo_embeddings = {}
reg_embeddings = {}
for i in range(len(vectors[0])):
    embedding = weights[0]*np.add(np.add(weights[1]*vectors[0][i], weights[2]*vectors[1][i]), weights[3]*vectors[2][i])
    elmo_embeddings[orig_sentence[i] + " " + str(i)] = (i, embedding)
    reg_embeddings[orig_sentence[i] + " " + str(i)] = (i, vectors[2][i])

    
def similarities(embedding_dict):
    cosine_sims = []
    for w1, (i, e1) in embedding_dict.items():
        for w2, (j, e2) in embedding_dict.items():
            if j <= i or w2.split()[0] == w1.split()[0]: 
                continue
            dist = scipy.spatial.distance.cosine(e1,e2)
            cosine_sims.append((i,j,w1.split()[0],w2.split()[0],dist))
    cosine_sims.sort(key=lambda x: x[4], reverse = False)
#     print(cosine_sims)
    return cosine_sims

top_n = 5

sim_elmo = similarities(elmo_embeddings)[:top_n]
sim_reg = similarities(reg_embeddings)[:top_n]

print(sim_elmo)

# print()

print(sim_reg)

# print(len(set([(a, b, c, d) for a, b, c, d, e in sim_elmo]).intersection(set([(a, b, c, d) for a, b, c, d, e in sim_reg]))))


[(33, 35, ',', 'and', 0.27203869819641113), (24, 47, 'international', 'International', 0.27933722734451294), (24, 83, 'international', 'International', 0.32194018363952637), (32, 34, 'hospitals', 'mosques', 0.34638386964797974), (98, 100, 'royally', 'comfortably', 0.3491635322570801)]
[(98, 100, 'royally', 'comfortably', 0.20597773790359497), (32, 34, 'hospitals', 'mosques', 0.20960217714309692), (16, 17, ',', 'because', 0.23645401000976562), (24, 47, 'international', 'International', 0.24221879243850708), (40, 78, 'protected', 'sheltered', 0.24946069717407227)]


In [6]:
sim_elmo_dict = {}
for i,j,w1,w2,dist in sim_elmo:
    sim_elmo_dict[str(i)+" "+str(j)] = (w1,w2,dist)

sim_reg_dict = {}
for i,j,w1,w2,dist in sim_reg:
    sim_reg_dict[str(i)+" "+str(j)] = (w1,w2,dist)    

In [7]:
for k,v in sim_elmo_dict.items():
    if (sim_elmo_dict[k][2] <= sim_reg_dict[k][2]):
#         pass
        print("Elmo <= Regular")
        print(sim_elmo_dict[k],sim_reg_dict[k])
    if (sim_elmo_dict[k][2] > sim_reg_dict[k][2]):
        pass
#         print("Elmo > Regular")
#         print(sim_elmo_dict[k],sim_reg_dict[k])

KeyError: '33 35'

In [8]:
weights

[1.016251564025879,
 0.027123697102069855,
 0.038384225219488144,
 -0.03157595172524452]

In [9]:
# Analyze the glove vectors
import numpy as np
import collections
import util
import sys
sys.argv[1] = 'trial'
config = util.initialize_from_env()
context_embeddings = util.EmbeddingDictionary(
    config["context_embeddings"])
head_embeddings = util.EmbeddingDictionary(
    config["head_embeddings"], maybe_cache=context_embeddings)

Setting CUDA_VISIBLE_DEVICES to: 
Running experiment: trial
max_top_antecedents = 50
max_training_sentences = 50
top_span_ratio = 0.4
filter_widths = [
  3
  4
  5
]
filter_size = 50
char_embedding_size = 8
char_vocab_path = "char_vocab.english.txt"
context_embeddings {
  path = "glove.840B.300d.txt"
  size = 300
}
head_embeddings {
  path = "glove_50_300_2.txt"
  size = 300
}
contextualization_size = 200
contextualization_layers = 3
ffnn_size = 150
ffnn_depth = 2
feature_size = 20
max_span_width = 30
use_metadata = true
use_features = true
model_heads = true
coref_depth = 2
lm_layers = 3
lm_size = 1024
coarse_to_fine = true
max_gradient_norm = 5.0
lstm_dropout_rate = 0.4
lexical_dropout_rate = 0.5
dropout_rate = 0.2
optimizer = "adam"
learning_rate = 0.001
decay_rate = 0.999
decay_frequency = 100
train_path = "train.english.jsonlines"
eval_path = "dev.english.jsonlines"
conll_eval_path = "dev.english.conll"
lm_path = ""
genres = [
  "bc"
  "bn"
  "mz"
  "nw"
  "pt"
  "tc"
  "wb"
]
eva

In [27]:
cosine_sims = []
head_embeddings = enumerate(head_embeddings._embeddings.items())
for i, (w1, e1) in head_embeddings:
    for j, (w2, e2) in head_embeddings:
        if i < j:
            dist = scipy.spatial.distance.cosine(e1,e2)
            cosine_sims.append((w1,w2,dist))

AttributeError: 'enumerate' object has no attribute '_embeddings'

In [13]:
cosine_sim_dict = {}
for w1,w2,dist in cosine_sims:
    cosine_sim_dict[w1 + " " + w2] = dist

In [19]:
for i,j,w1,w2,dist in sim_reg:
    w1 = w1.lower()
    w2 = w2.lower()
    if w1 + ' ' + w2 in cosine_sim_dict:
        print(w1 + ' ' + w2, cosine_sim_dict[w1 + ' ' + w2])
    elif w2 + ' ' + w1 in cosine_sim_dict:
        print(w1 + ' ' + w2, cosine_sim_dict[w2 + ' ' + w1])

, because 0.45479732751846313


In [26]:
words_elmo = []
words_dict = []
for i, j, w1, w2, _ in sim_elmo + sim_reg:
    words_elmo.append(w1)
    words_elmo.append(w2)
    
for key in cosine_sim_dict:
    k = key.split(" ")
#     print(k)
    words_dict.append(k[0])
    words_dict.append(k[1])
    
print((set(words_elmo).intersection(set(words_dict))))

{'international', ',', 'and', 'International', 'because'}
